# Preprocessamento

In [ ]:
!pip install geocoder
import pandas as pd
import numpy as np
import geocoder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 98 kB 4.3 MB/s 


In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/tiagodavi70/azulejos_vis/master/TabelasAzulejo.csv", delimiter=";")

In [ ]:
# Formatar os valores nulos
for coluna in df.columns[1:]:
    df[coluna] = df[coluna].where(df[coluna] != "x", "")

In [ ]:
# Formatação dos tamanhos doz azulejos para converter de strign para números em altura e largura
dimensoes = df["Dimensão-Formato"].str.replace(" ", "").str.replace("cm", "").str.replace(",", ".").str.extract(r"(?P<altura>[+-]?([0-9]*[.])?[0-9]+)x(?P<largura>[+-]?([0-9]*[.])?[0-9]+)")
df["Altura"] = dimensoes["altura"]
df["Largura"] = dimensoes["largura"]

In [ ]:
endereco_serie = df["Endereço"].where(~ df["Endereço"].str.contains("Demolido"), "DEMOLIDO")

df_completo = []

for (i, end) in enumerate(endereco_serie):
    if (end != "DEMOLIDO"):
        ends = end.split(";")
        # print(ends)
        for item in ends:
            divisa = item.split("n°")
            logradouroSN = divisa[0]
            if (len(divisa) > 1):
                numeros = divisa[1].strip()
                # laco para repetir numero por numero do endereco (mais generico)
                numeros = numeros.replace(",", " ").replace("/", " ").replace(" e ", " ")
                numeros_processados = []
                for num in numeros.split(" "):
                    if num.isnumeric():
                        numeros_processados.append(num)
                    else:
                        break
                for numero_atual in numeros_processados:
                    lista_numeros = numero_atual.replace("e", ",").split(" ") 
                    # completar aqui também com os vários endereços
                    completo = logradouroSN.strip() + " n " + numero_atual + ", Belém, Pará, Brasil"
                    if (completo != ""):
                        # print(completo)
                        new_cell = df.loc[i].copy()
                        g = geocoder.osm(completo)
                        if (g.latlng):
                            lat, lng = g.latlng[0], g.latlng[1]
                            new_cell["latitude"] = lat
                            new_cell["longitude"] = lng
                            new_cell["Endereço"] = completo
                            df_completo.append(new_cell)

df = pd.DataFrame(df_completo)

In [ ]:
df

,Azulejo-Código,Procedência,Fábrica,Dimensão-Formato,"Material, Técnica de Fabricação e Decorativa",Endereço,Bairro,Uso-Aplicação,Altura,Largura,latitude,longitude
0,Azulejo-PE 1-1-1,Lisboa/Portugal,Viúva Lamego-Constância (Consta do Catalogo de...,"13,4 x 13,4cm",Cerâmica e vidrado Estampilha,"Travessa Frutuoso Guimarães n 18, Belém, Pará,...",Cidade Velha/Campina,"Revestimento de fachada, assentados por compre...",13.4,13.4,-1.451037,-48.501079
0,Azulejo-PE 1-1-1,Lisboa/Portugal,Viúva Lamego-Constância (Consta do Catalogo de...,"13,4 x 13,4cm",Cerâmica e vidrado Estampilha,"Travessa Frutuoso Guimarães n 30, Belém, Pará,...",Cidade Velha/Campina,"Revestimento de fachada, assentados por compre...",13.4,13.4,-1.451037,-48.501079
0,Azulejo-PE 1-1-1,Lisboa/Portugal,Viúva Lamego-Constância (Consta do Catalogo de...,"13,4 x 13,4cm",Cerâmica e vidrado Estampilha,"Travessa Frutuoso Guimarães n 36, Belém, Pará,...",Cidade Velha/Campina,"Revestimento de fachada, assentados por compre...",13.4,13.4,-1.451037,-48.501079
0,Azulejo-PE 1-1-1,Lisboa/Portugal,Viúva Lamego-Constância (Consta do Catalogo de...,"13,4 x 13,4cm",Cerâmica e vidrado Estampilha,"Travessa Gurupá n 114, Belém, Pará, Brasil",Cidade Velha/Campina,"Revestimento de fachada, assentados por compre...",13.4,13.4,-1.459677,-48.502419
0,Azulejo-PE 1-1-1,Lisboa/Portugal,Viúva Lamego-Constância (Consta do Catalogo de...,"13,4 x 13,4cm",Cerâmica e vidrado Estampilha,"Travessa Gurupá n 266, Belém, Pará, Brasil",Cidade Velha/Campina,"Revestimento de fachada, assentados por compre...",13.4,13.4,-1.459677,-48.502419
...,...,...,...,...,...,...,...,...,...,...,...,...
269,Azulejo-Plq 1 (Branco),,,"14,6 x 14,6 cm",Cerâmica e vidrado Liso,"Avenida Conselheiro Furtado n 1881, Belém, Par...",Campina/Nazaré,"Revestimento de fachada, assentados por compre...",14.6,14.6,-1.456446,-48.480971
269,Azulejo-Plq 1 (Branco),,,"14,6 x 14,6 cm",Cerâmica e vidrado Liso,"Avenida Governador José Malcher n 459, Belém, ...",Campina/Nazaré,"Revestimento de fachada, assentados por compre...",14.6,14.6,-1.452029,-48.493545
270,Azulejo-Plq 2 (Verde),,Servais Werke A.G. Ehrang,15 x 15 cm,Cerâmica e vidrado Liso,"Rua Gama Abreu n 1190, Belém, Pará, Brasil",Batista Campos,"Revestimento de fachada, assentados por compre...",15,15,-1.456543,-48.495189
270,Azulejo-Plq 2 (Verde),,Servais Werke A.G. Ehrang,15 x 15 cm,Cerâmica e vidrado Liso,"Rua Gama Abreu n 1202, Belém, Pará, Brasil",Batista Campos,"Revestimento de fachada, assentados por compre...",15,15,-1.456487,-48.495109


In [ ]:
df.to_csv("TabelasAzulejoEditadaEnderecos.csv", index=None)